In [ ]:
!pip install -r requirements.txt

Install tesseract OCR from https://github.com/tesseract-ocr/tesseract

In [ ]:
!pip install pytesseract

In [ ]:
!python detect.py --weights best.pt --source test --conf 0.25 --save-txt --save-conf

Before executing the below cell
Replace images_path with the YOLO output folder created while executing the previous cell

In [ ]:
import os
import cv2
import pytesseract
import re

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
images_path = 'runs/detect/exp/'  # Replace with your YOLO output folder
labels_path = os.path.join(images_path, 'labels')

ocr_config = '--psm 7'

for label_file in os.listdir(labels_path):
    if label_file.endswith('.txt'):
        image_file = label_file.replace('.txt', '.jpg') 
        image_path = os.path.join(images_path, image_file)

        img = cv2.imread(image_path)
        if img is None:
            print(f"Error: Image {image_file} not found.")
            continue

        with open(os.path.join(labels_path, label_file), 'r') as f:
            for line in f:
                _, x, y, w, h, _ = map(float, line.split())

                h_img, w_img, _ = img.shape
                x1 = int((x - w / 2) * w_img)
                y1 = int((y - h / 2) * h_img)
                x2 = int((x + w / 2) * w_img)
                y2 = int((y + h / 2) * h_img)

                crop = img[y1:y2, x1:x2]

                text = pytesseract.image_to_string(crop, config=ocr_config).strip()

                cleaned_text = re.sub(r'[^A-Za-z0-9]', '', text)

                print(f"Detected text for {image_file}: {cleaned_text}")